In [1]:
import pandas
import pickle
import os
import pandas as pd
import nltk
import json
from datetime import datetime, timezone, date, time

In [2]:
STOCK_DATA_DIR = '/media/adv/Data/PROJECTS/CSE573-SWM/Stock Data/'
TIME_FRAMES = ['30-min', '1-hr', '4-hr', '1-day']

CURRENT_TIMEFRAME = '1-day'

In [3]:
# Train data on 1 hour

if CURRENT_TIMEFRAME == '1-day':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE1440.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON1440.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '4-hr':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE240.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON240.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '1-hr':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE60.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON60.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '30-min':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE30.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON30.csv'),names=['date','time','open','high','low','close','vol'])


In [4]:
# Label the stock price direction as -1, 1
for df in [aapl_stock_prices, amzn_stock_prices]:
    df['diff'] = (df['open'] <= df['close'])
    df['diff'] = df['diff'].apply(int)
    df['diff'].loc[df['diff']==0] = -1
    df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y.%m.%d').date())
    df['time'] = df['time'].apply(lambda x: datetime.strptime(x, '%H:%M').time())
    df['timestamp'] = df.apply(lambda x: datetime.combine(x['date'], x['time']), 1)
    df.sort_values(['timestamp'], axis=0, ascending=True, inplace=True)
    df.drop(columns=['date', 'time'], inplace=True)
    
aapl_stock_prices

/home/adv/Tools/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,open,high,low,close,vol,diff,timestamp
0,93.159,94.194,93.028,93.460,2799,1,2014-06-30
1,93.973,94.546,93.611,93.993,4787,1,2014-07-01
2,94.344,94.515,93.540,93.902,4476,-1,2014-07-02
3,94.113,94.545,93.661,94.495,3454,1,2014-07-03
4,94.936,96.423,94.936,96.403,4638,1,2014-07-07
...,...,...,...,...,...,...,...
1138,156.250,156.490,153.650,155.920,62867,-1,2019-01-28
1139,156.470,161.620,150.980,160.540,66595,1,2019-01-29
1140,161.680,166.110,160.220,165.660,70344,1,2019-01-30
1141,166.100,168.980,164.530,166.770,62821,1,2019-01-31


In [5]:
# Load tokenized news data


with open('AaplExtractedTokens.pkl', 'rb') as f:
    tokenized_aapl_news_df = pickle.load(f)
    
with open('AmznExtractedTokens.pkl', 'rb') as f:
    tokenized_amzn_news_df = pickle.load(f)

In [6]:
# Join news data and stock direction to label news
columns = ['news_timestamp', 'stock_timestamp', 'source', 'tokens', 'label']
labelled_aapl_news = pd.DataFrame(columns=columns)
labelled_amzn_news = pd.DataFrame(columns=columns)

skipped_cnt = 0
for index, row in tokenized_aapl_news_df.iterrows():
    try:
        matching_row = aapl_stock_prices.loc[aapl_stock_prices['timestamp'] > row['timestamp']].reset_index(drop=True).loc[0]
        labelled_aapl_news.loc[len(labelled_aapl_news)] = [
            row['timestamp'],
            matching_row['timestamp'],
            row['source'],
            row['tokens'],
            matching_row['diff'],
        ]
    except:
        skipped_cnt += 1
print("Skipped %d apple news articles with corresponding stock chart missing" % skipped_cnt)

skipped_cnt = 0
for index, row in tokenized_amzn_news_df.iterrows():
    try:
        matching_row = amzn_stock_prices.loc[amzn_stock_prices['timestamp'] > row['timestamp']].reset_index(drop=True).loc[0]
        labelled_amzn_news.loc[len(labelled_amzn_news)] = [
            row['timestamp'],
            matching_row['timestamp'],
            row['source'],
            row['tokens'],
            matching_row['diff'],
        ]
    except:
        skipped_cnt += 1
print("Skipped %d amazon news articles with corresponding stock chart missing" % skipped_cnt)

Skipped 1234 apple news articles with corresponding stock chart missing
Skipped 320 amazon news articles with corresponding stock chart missing


In [7]:
# Save labelled news data

with open('AmznLabelledNewsData.pkl', 'wb') as f:
    pickle.dump(labelled_amzn_news, f)
with open('AaplLabelledNewsData.pkl', 'wb') as f:
    pickle.dump(labelled_aapl_news, f)

In [8]:
labelled_aapl_news

,news_timestamp,stock_timestamp,source,tokens,label
0,2017-12-07 20:00:00,2017-12-08,businesswire.com,"[nape, summit, also, annual, luncheon, tom, sp...",-1
1,2017-12-08 21:37:00,2017-12-11,yahoo.com,"[huge, x, comeback, tour, follow, note, big, y...",1
2,2017-12-12 01:55:00,2017-12-13,yahoo.com,"[browser, os, ad, server, make, first, move, c...",1
3,2017-12-12 22:10:00,2017-12-13,investingnews.com,"[humanitarian, place, conflict, free, region]",1
4,2017-12-14 11:42:00,2017-12-15,seekingalpha.com,"[prefer, invest]",1
...,...,...,...,...,...
71936,2019-01-31 23:24:00,2019-02-01,marketwatch.com,"[deem, store]",1
71937,2019-01-31 23:31:00,2019-02-01,marketwatch.com,"[deem, use, key, develop, tool, day, later]",1
71938,2019-01-31 23:52:00,2019-02-01,tickerreport.com,"[tweet, trader, share, trade, tuesday, stock, ...",1
71939,2019-01-31 23:53:00,2019-02-01,redliontrader.com,"[obtain, intern, memo, tri, spin, led, block, ...",1


In [9]:
labelled_amzn_news

,news_timestamp,stock_timestamp,source,tokens,label
0,2017-12-15 12:04:00,2017-12-18,seekingalpha.com,"[amazon, soon, sell, pull, e, platform, two, y...",1
1,2017-12-21 12:04:00,2017-12-22,seekingalpha.com,"[line, cashier, similar, fashion, futurist, am...",-1
2,2017-12-21 12:27:00,2017-12-22,thestreet.com,"[fit, fang, stock, amazon, alphabet, drug, int...",-1
3,2017-12-22 00:00:00,2017-12-26,investors.com,"[amazon, cloud, china]",1
4,2017-12-22 20:57:00,2017-12-26,yahoo.com,"[stream, stick, plus, offer, play, amazon, con...",1
...,...,...,...,...,...
19911,2019-01-31 21:42:00,2019-02-01,nasdaq.com,"[also, prove, reach, mani, entertain, age, soo...",-1
19912,2019-01-31 22:03:00,2019-02-01,makemefeed.com,"[read, amazon, general, avail, publish, cost, ...",-1
19913,2019-01-31 22:05:00,2019-02-01,yahoo.com,"[direct, cloud, amazon, unit, driven, match, l...",-1
19914,2019-01-31 23:24:00,2019-02-01,marketwatch.com,"[earn, up, say, spend, strong, amazon, real, t...",-1
